## Session Length Prediction -- Beat the professors:)

In [463]:
import os
import lxml

import pandas as pd

#### Use maximum width of notebook

In [464]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_colwidth', 500)
pd.set_option('expand_frame_repr', True)

In [465]:
os.chdir("/data_data/session_length/")
!pwd

/data_data/session_length


### Make sure no other spark jobs running before doing this

In [466]:
#import pymongo
#import pymongo_spark

#pymongo_spark.activate()


In [467]:
#sc.stop()
#spark.stop()

### If you want to restart Yarn etc

In [468]:
#!/usr/local/hadoop/sbin/stop-all.sh
#!/usr/local/hadoop/sbin/start-all.sh

In [470]:
APP_NAME = "Session Length Prediction"

try:
    sc.stop()
    spark.stop()
except:
    pass

from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

try:
    sc and spark
except (NameError, UnboundLocalError) as e:
    import findspark
    #findspark.init()
    import pyspark
    import pyspark.sql
    
    #sc = pyspark.SparkContext()
    #spark = pyspark.sql.SparkSession(sc).builder.appName(APP_NAME).getOrCreate()
    
sc = SparkContext(conf = SparkConf() .set("spark.driver.maxResultSize", "40g") .set("spark.sql.execution.arrow.enabled", "true") .set('spark.sql.broadcastTimeout', 1000) .set('spark.local.dir', '/data_data/session_length/spark_tmp/') .set('spark.driver.memory', '80G') .set("spark.executor.instances", "20") .set("spark.executor.cores", 20) .set("spark.executor.memory", "12G")).getOrCreate()
spark = SparkSession(sc)
spark.sparkContext.setLogLevel("ERROR")
    

### Check configuration of Spark Environment

In [471]:
sc._conf.getAll()

[(u'spark.local.dir', u'/data_data/session_length/spark_tmp/'),
 (u'spark.sql.execution.arrow.enabled', u'true'),
 (u'spark.executor.cores', u'20'),
 (u'spark.driver.memory', u'80G'),
 (u'spark.executor.id', u'driver'),
 (u'spark.app.name', u'pyspark-shell'),
 (u'spark.executor.instances', u'20'),
 (u'spark.driver.port', u'35233'),
 (u'spark.driver.maxResultSize', u'40g'),
 (u'spark.rdd.compress', u'True'),
 (u'spark.app.id', u'local-1537128781907'),
 (u'spark.driver.host', u'ip-172-31-41-62.ec2.internal'),
 (u'spark.serializer.objectStreamReset', u'100'),
 (u'spark.master', u'local[*]'),
 (u'spark.submit.deployMode', u'client'),
 (u'spark.sql.broadcastTimeout', u'1000'),
 (u'spark.executor.memory', u'12G'),
 (u'spark.ui.showConsoleProgress', u'true')]

In [472]:
from pyspark.sql.types import StructField, StructType, StringType, LongType, DateType, DoubleType, IntegerType
from pyspark.sql.functions import count, mean, stddev_pop, min, max, lit, round, bround, pow, col, corr, lower, upper, avg, stddev, abs, log
from pyspark.sql.functions import lit, trim, rtrim, rpad, trim, coalesce
from pyspark.sql.functions import current_date, current_timestamp, date_add, date_sub, months_between, to_date
from pyspark.sql.functions import udf, col, sum
from pyspark.sql.window import Window
from pyspark.sql.functions import desc, dense_rank, rank, expr

from pyspark.ml import Pipeline
from pyspark.ml.feature import RFormula
from pyspark.ml.classification import LogisticRegression, GBTClassifier, DecisionTreeClassifier, RandomForestClassifier
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from datetime import datetime

### Import Data

#### Import utility tools

#### Import both data sets

In [473]:
!head data/lastFM/lastfm-dataset-1K/userid-profile.tsv

#id	gender	age	country	registered
user_000001	m		Japan	Aug 13, 2006
user_000002	f		Peru	Feb 24, 2006
user_000003	m	22	United States	Oct 30, 2005
user_000004	f			Apr 26, 2006
user_000005	m		Bulgaria	Jun 29, 2006
user_000006		24	Russian Federation	May 18, 2006
user_000007	f		United States	Jan 22, 2006
user_000008	m	23	Slovakia	Sep 28, 2006
user_000009	f	19	United States	Jan 13, 2007


In [474]:
users_df = None

In [475]:
from_pattern = 'MMM dd, yyyy'
to_pattern = 'yyyy-MM-dd'

def import_user_data():
    global users_df
    print("==================================================================================================================")
    print("======================================== IMPORTING USERID-PROFILE.CSV  ======================================")
    print("==================================================================================================================\n\n")
    userSchema = StructType([\
        StructField('userid',  StringType(), True),\
        StructField('gender',  StringType(), True),\
        StructField('age',     IntegerType(), True),\
        StructField('country', StringType(), True),\
        StructField('registered', StringType(), True)])
    users = spark.read.format('csv').schema(userSchema).option("sep","\t").load('/data_data/session_length/data/lastFM/lastfm-dataset-1K/userid-profile.tsv')
    users.show(5,False)
    #func = udf(lambda x: datetime.strptime(x, to_pattern), DateType() )
    users_df = users.withColumn('reg_date', to_date(col("registered"), from_pattern))
    users_df.show(5,False)

In [476]:
import_user_data()

======================================== IMPORTING USERID-PROFILE.CSV  ======================================


+-----------+------+----+-------------+------------+
|userid     |gender|age |country      |registered  |
+-----------+------+----+-------------+------------+
|null       |null  |null|null         |null        |
|user_000001|m     |null|Japan        |Aug 13, 2006|
|user_000002|f     |null|Peru         |Feb 24, 2006|
|user_000003|m     |22  |United States|Oct 30, 2005|
|user_000004|f     |null|null         |Apr 26, 2006|
+-----------+------+----+-------------+------------+
only showing top 5 rows

+-----------+------+----+-------------+------------+----------+
|userid     |gender|age |country      |registered  |reg_date  |
+-----------+------+----+-------------+------------+----------+
|null       |null  |null|null         |null        |null      |
|user_000001|m     |null|Japan        |Aug 13, 2006|2006-08-13|
|user_000002|f     |null|Peru         |Feb 24, 2006|2006-02-24|
|u

#### Import music_data

In [477]:
!head data/lastFM/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv

user_000001	2009-05-04T23:08:57Z	f1b1cf71-bd35-4e99-8624-24a6e15f133a	Deep Dish		Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
user_000001	2009-05-04T13:54:10Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Composition 0919 (Live_2009_4_15)
user_000001	2009-05-04T13:52:04Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc2 (Live_2009_4_15)
user_000001	2009-05-04T13:42:52Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Hibari (Live_2009_4_15)
user_000001	2009-05-04T13:42:11Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc1 (Live_2009_4_15)
user_000001	2009-05-04T13:38:31Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		To Stanford (Live_2009_4_15)
user_000001	2009-05-04T13:33:28Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Improvisation (Live_2009_4_15)
user_000001	2009-05-04T13:23:45Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Glacier (Live_2009_4_15)
user_000001	2009-05-04T13:19:22Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Parolibre (Live_2009_4_15)
user_000001	2009-05-04T13:13:38Z	a7f7df4a-77d8-4f12-8acd

In [478]:
sessions_DF = None
sessions_df = None

In [479]:
from_pattern = 'yyyy-MM-ddTHH:mm:ssZ'
to_pattern = 'yyyy-MM-dd'

def import_session_data():
    global sessions_DF
    global sessions_df
    print("====================================================================================================================")
    print("====================================userid-timestamp-artid-artname-traid-traname.tsv================================")
    print("==================================================================================================================\n\n")
    userSchema = StructType([\
        StructField('userid',  StringType(), True),\
        StructField('timestamp',  StringType(), True),\
        StructField('artid',     StringType(), True),\
        StructField('artname_', StringType(), True),\
        StructField('traid', StringType(), True),\
        StructField('traname_', StringType(), True)
        ])
    sessions = spark.read.format('csv').schema(userSchema).option("sep","\t").load('/data_data/session_length/data/lastFM/lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv')
    sessions.show(5,False)
    #Assign window to time (using hour as window....)
    def get_window(x):
        return int(str(x).split("T")[1].split(":")[0])
    getWindow = udf(lambda timestamp:  get_window(timestamp))
    sessions_DF = sessions.withColumn('session_date',to_date(col('timestamp'))).withColumn('session_window', getWindow(col('timestamp')))
    sessions_DF.show(5,False)
    #Translate artname_ to English
    def translate_to_english(str_to_translate):
        return gs.translate(str_to_translate, 'en')
    func = udf(lambda str_to_translate:  translate_to_english(str_to_translate), StringType())
    #sessions_df = sessions_DF.withColumn('artname', translate_to_english('artname_'))
    #sessions_df[["artname_"]].apply(lambda row: np.mean(row),axis=0)
    #sessions_df["artname"] = sessions_DF["artname_"].apply(lambda name: translate_to_english(name),axis=0)
    #.withColumn('traname', func(col('traname_')))
    sessions_df = sessions_DF.rdd.map(lambda x: (x["artname_"]))
    print(sessions_df.take(5))

In [480]:
import_session_data()

====================================userid-timestamp-artid-artname-traid-traname.tsv================================


+-----------+--------------------+------------------------------------+---------+-----+------------------------------------------+
|userid     |timestamp           |artid                               |artname_ |traid|traname_                                  |
+-----------+--------------------+------------------------------------+---------+-----+------------------------------------------+
|user_000001|2009-05-04T23:08:57Z|f1b1cf71-bd35-4e99-8624-24a6e15f133a|Deep Dish|null |Fuck Me Im Famous (Pacha Ibiza)-09-28-2007|
|user_000001|2009-05-04T13:54:10Z|a7f7df4a-77d8-4f12-8acd-5c60c93f4de8|坂本龍一     |null |Composition 0919 (Live_2009_4_15)         |
|user_000001|2009-05-04T13:52:04Z|a7f7df4a-77d8-4f12-8acd-5c60c93f4de8|坂本龍一     |null |Mc2 (Live_2009_4_15)                      |
|user_000001|2009-05-04T13:42:52Z|a7f7df4a-77d8-4f12-8acd-5c60c93f4de8|坂本龍一     |null |Hibari (

In [481]:
#from pyspark.sql.types import Row

#row = Row("artname_")
#df_sessions = sessions_df.map(row).toDF()
#schema = StructType([StructField("artname", StringType(), True)])
#df_sessions = sessions_df.toDF(["artname"],)
#[gs.translate(x,'en') for x in df_sessions]
#df_sessions = sessions_df.flatMap(gs.translate)
#df_sessions.take(5)

In [482]:
users_df.show()

+-----------+------+----+------------------+------------+----------+
|     userid|gender| age|           country|  registered|  reg_date|
+-----------+------+----+------------------+------------+----------+
|       null|  null|null|              null|        null|      null|
|user_000001|     m|null|             Japan|Aug 13, 2006|2006-08-13|
|user_000002|     f|null|              Peru|Feb 24, 2006|2006-02-24|
|user_000003|     m|  22|     United States|Oct 30, 2005|2005-10-30|
|user_000004|     f|null|              null|Apr 26, 2006|2006-04-26|
|user_000005|     m|null|          Bulgaria|Jun 29, 2006|2006-06-29|
|user_000006|  null|  24|Russian Federation|May 18, 2006|2006-05-18|
|user_000007|     f|null|     United States|Jan 22, 2006|2006-01-22|
|user_000008|     m|  23|          Slovakia|Sep 28, 2006|2006-09-28|
|user_000009|     f|  19|     United States|Jan 13, 2007|2007-01-13|
|user_000010|     m|  19|            Poland| May 4, 2006|2006-05-04|
|user_000011|     m|  21|         

#### Denormalize by joining user and session databases

In [483]:
users_df = users_df.withColumnRenamed('userid','user_id')
u = users_df.alias('u')
s = sessions_DF.alias('s')
join_condition = [ (u.user_id == s.userid) ]
sessionsDF = s.join(u, join_condition, 'inner').drop('user_id')
sessionsDF.show(20,False)

+-----------+--------------------+------------------------------------+---------------+------------------------------------+------------------------------------------+------------+--------------+------+----+-------+------------+----------+
|userid     |timestamp           |artid                               |artname_       |traid                               |traname_                                  |session_date|session_window|gender|age |country|registered  |reg_date  |
+-----------+--------------------+------------------------------------+---------------+------------------------------------+------------------------------------------+------------+--------------+------+----+-------+------------+----------+
|user_000001|2009-05-04T23:08:57Z|f1b1cf71-bd35-4e99-8624-24a6e15f133a|Deep Dish      |null                                |Fuck Me Im Famous (Pacha Ibiza)-09-28-2007|2009-05-04  |23            |m     |null|Japan  |Aug 13, 2006|2006-08-13|
|user_000001|2009-05-04T13:54:10Z|a7f7df

#### Safe as TSV file for Time Series analysis

#### Uncomment if you want a fresh copy of the file

In [484]:
#!rm -R /data_data/session_length/data/tmp
#sessionsDF.coalesce(1).write.option("delimiter", "\t").csv('/data_data/session_length/data/tmp', header="True")
#!mv /data_data/session_length/data/tmp/part-* /data_data/session_length/data/Session_data.csv

In [485]:
!head /data_data/session_length/data/Session_data.csv

userid	timestamp	artid	artname_	traid	traname_	session_window	gender	age	country	registered	reg_date
user_000001	2009-05-04T23:08:57Z	f1b1cf71-bd35-4e99-8624-24a6e15f133a	Deep Dish		Fuck Me Im Famous (Pacha Ibiza)-09-28-2007	23	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:54:10Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Composition 0919 (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:52:04Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc2 (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:42:52Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Hibari (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:42:11Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		Mc1 (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:38:31Z	a7f7df4a-77d8-4f12-8acd-5c60c93f4de8	坂本龍一		To Stanford (Live_2009_4_15)	13	m		Japan	Aug 13, 2006	2006-08-13
user_000001	2009-05-04T13:33:28Z	a7f7df4a-77d8-4f1

### Get Local time by converting from UTC to Local [WORK IN PROGRESS]

In [487]:
from dateutil import tz
import pytz

### Use countryInfo from here: https://gist.githubusercontent.com/pamelafox/986163/raw/f5f9db4f1b287804fd07ffb3296ed0036292bc7a/countryinfo.py

In [488]:
countries_info = [
{'timezones': ['Europe/Andorra'], 'code': 'AD', 'continent': 'Europe', 'name': 'Andorra', 'capital': 'Andorra la Vella'},
{'timezones': ['Asia/Kabul'], 'code': 'AF', 'continent': 'Asia', 'name': 'Afghanistan', 'capital': 'Kabul'},
{'timezones': ['America/Antigua'], 'code': 'AG', 'continent': 'North America', 'name': 'Antigua and Barbuda', 'capital': "St. John's"},
{'timezones': ['Europe/Tirane'], 'code': 'AL', 'continent': 'Europe', 'name': 'Albania', 'capital': 'Tirana'},
{'timezones': ['Asia/Yerevan'], 'code': 'AM', 'continent': 'Asia', 'name': 'Armenia', 'capital': 'Yerevan'},
{'timezones': ['Africa/Luanda'], 'code': 'AO', 'continent': 'Africa', 'name': 'Angola', 'capital': 'Luanda'},
{'timezones': ['America/Argentina/Buenos_Aires', 'America/Argentina/Cordoba', 'America/Argentina/Jujuy', 'America/Argentina/Tucuman', 'America/Argentina/Catamarca', 'America/Argentina/La_Rioja', 'America/Argentina/San_Juan', 'America/Argentina/Mendoza', 'America/Argentina/Rio_Gallegos', 'America/Argentina/Ushuaia'], 'code': 'AR', 'continent': 'South America', 'name': 'Argentina', 'capital': 'Buenos Aires'},
{'timezones': ['Europe/Vienna'], 'code': 'AT', 'continent': 'Europe', 'name': 'Austria', 'capital': 'Vienna'},
{'timezones': ['Australia/Lord_Howe', 'Australia/Hobart', 'Australia/Currie', 'Australia/Melbourne', 'Australia/Sydney', 'Australia/Broken_Hill', 'Australia/Brisbane', 'Australia/Lindeman', 'Australia/Adelaide', 'Australia/Darwin', 'Australia/Perth'], 'code': 'AU', 'continent': 'Oceania', 'name': 'Australia', 'capital': 'Canberra'},
{'timezones': ['Asia/Baku'], 'code': 'AZ', 'continent': 'Asia', 'name': 'Azerbaijan', 'capital': 'Baku'},
{'timezones': ['America/Barbados'], 'code': 'BB', 'continent': 'North America', 'name': 'Barbados', 'capital': 'Bridgetown'},
{'timezones': ['Asia/Dhaka'], 'code': 'BD', 'continent': 'Asia', 'name': 'Bangladesh', 'capital': 'Dhaka'},
{'timezones': ['Europe/Brussels'], 'code': 'BE', 'continent': 'Europe', 'name': 'Belgium', 'capital': 'Brussels'},
{'timezones': ['Africa/Ouagadougou'], 'code': 'BF', 'continent': 'Africa', 'name': 'Burkina Faso', 'capital': 'Ouagadougou'},
{'timezones': ['Europe/Sofia'], 'code': 'BG', 'continent': 'Europe', 'name': 'Bulgaria', 'capital': 'Sofia'},
{'timezones': ['Asia/Bahrain'], 'code': 'BH', 'continent': 'Asia', 'name': 'Bahrain', 'capital': 'Manama'},
{'timezones': ['Africa/Bujumbura'], 'code': 'BI', 'continent': 'Africa', 'name': 'Burundi', 'capital': 'Bujumbura'},
{'timezones': ['Africa/Porto-Novo'], 'code': 'BJ', 'continent': 'Africa', 'name': 'Benin', 'capital': 'Porto-Novo'},
{'timezones': ['Asia/Brunei'], 'code': 'BN', 'continent': 'Asia', 'name': 'Brunei Darussalam', 'capital': 'Bandar Seri Begawan'},
{'timezones': ['America/La_Paz'], 'code': 'BO', 'continent': 'South America', 'name': 'Bolivia', 'capital': 'Sucre'},
{'timezones': ['America/Noronha', 'America/Belem', 'America/Fortaleza', 'America/Recife', 'America/Araguaina', 'America/Maceio', 'America/Bahia', 'America/Sao_Paulo', 'America/Campo_Grande', 'America/Cuiaba', 'America/Porto_Velho', 'America/Boa_Vista', 'America/Manaus', 'America/Eirunepe', 'America/Rio_Branco'], 'code': 'BR', 'continent': 'South America', 'name': 'Brazil', 'capital': 'Bras\xc3\xadlia'},
{'timezones': ['America/Nassau'], 'code': 'BS', 'continent': 'North America', 'name': 'Bahamas', 'capital': 'Nassau'},
{'timezones': ['Asia/Thimphu'], 'code': 'BT', 'continent': 'Asia', 'name': 'Bhutan', 'capital': 'Thimphu'},
{'timezones': ['Africa/Gaborone'], 'code': 'BW', 'continent': 'Africa', 'name': 'Botswana', 'capital': 'Gaborone'},
{'timezones': ['Europe/Minsk'], 'code': 'BY', 'continent': 'Europe', 'name': 'Belarus', 'capital': 'Minsk'},
{'timezones': ['America/Belize'], 'code': 'BZ', 'continent': 'North America', 'name': 'Belize', 'capital': 'Belmopan'},
{'timezones': ['America/St_Johns', 'America/Halifax', 'America/Glace_Bay', 'America/Moncton', 'America/Goose_Bay', 'America/Blanc-Sablon', 'America/Montreal', 'America/Toronto', 'America/Nipigon', 'America/Thunder_Bay', 'America/Pangnirtung', 'America/Iqaluit', 'America/Atikokan', 'America/Rankin_Inlet', 'America/Winnipeg', 'America/Rainy_River', 'America/Cambridge_Bay', 'America/Regina', 'America/Swift_Current', 'America/Edmonton', 'America/Yellowknife', 'America/Inuvik', 'America/Dawson_Creek', 'America/Vancouver', 'America/Whitehorse', 'America/Dawson'], 'code': 'CA', 'continent': 'North America', 'name': 'Canada', 'capital': 'Ottawa'},
{'timezones': ['Africa/Kinshasa', 'Africa/Lubumbashi'], 'code': 'CD', 'continent': 'Africa', 'name': 'Democratic Republic of the Congo', 'capital': 'Kinshasa'},
{'timezones': ['Africa/Brazzaville'], 'code': 'CG', 'continent': 'Africa', 'name': 'Republic of the Congo', 'capital': 'Brazzaville'},
{'timezones': ['Africa/Abidjan'], 'code': 'CI', 'continent': 'Africa', 'name': "C\xc3\xb4te d'Ivoire", 'capital': 'Yamoussoukro'},
{'timezones': ['America/Santiago', 'Pacific/Easter'], 'code': 'CL', 'continent': 'South America', 'name': 'Chile', 'capital': 'Santiago'},
{'timezones': ['Africa/Douala'], 'code': 'CM', 'continent': 'Africa', 'name': 'Cameroon', 'capital': 'Yaound\xc3\xa9'},
{'timezones': ['Asia/Shanghai', 'Asia/Harbin', 'Asia/Chongqing', 'Asia/Urumqi', 'Asia/Kashgar'], 'code': 'CN', 'continent': 'Asia', 'name': "People's Republic of China", 'capital': 'Beijing'},
{'timezones': ['America/Bogota'], 'code': 'CO', 'continent': 'South America', 'name': 'Colombia', 'capital': 'Bogot\xc3\xa1'},
{'timezones': ['America/Costa_Rica'], 'code': 'CR', 'continent': 'North America', 'name': 'Costa Rica', 'capital': 'San Jos\xc3\xa9'},
{'timezones': ['America/Havana'], 'code': 'CU', 'continent': 'North America', 'name': 'Cuba', 'capital': 'Havana'},
{'timezones': ['Atlantic/Cape_Verde'], 'code': 'CV', 'continent': 'Africa', 'name': 'Cape Verde', 'capital': 'Praia'},
{'timezones': ['Asia/Nicosia'], 'code': 'CY', 'continent': 'Asia', 'name': 'Cyprus', 'capital': 'Nicosia'},
{'timezones': ['Europe/Prague'], 'code': 'CZ', 'continent': 'Europe', 'name': 'Czech Republic', 'capital': 'Prague'},
{'timezones': ['Europe/Berlin'], 'code': 'DE', 'continent': 'Europe', 'name': 'Germany', 'capital': 'Berlin'},
{'timezones': ['Africa/Djibouti'], 'code': 'DJ', 'continent': 'Africa', 'name': 'Djibouti', 'capital': 'Djibouti City'},
{'timezones': ['Europe/Copenhagen'], 'code': 'DK', 'continent': 'Europe', 'name': 'Denmark', 'capital': 'Copenhagen'},
{'timezones': ['America/Dominica'], 'code': 'DM', 'continent': 'North America', 'name': 'Dominica', 'capital': 'Roseau'},
{'timezones': ['America/Santo_Domingo'], 'code': 'DO', 'continent': 'North America', 'name': 'Dominican Republic', 'capital': 'Santo Domingo'},
{'timezones': ['America/Guayaquil', 'Pacific/Galapagos'], 'code': 'EC', 'continent': 'South America', 'name': 'Ecuador', 'capital': 'Quito'},
{'timezones': ['Europe/Tallinn'], 'code': 'EE', 'continent': 'Europe', 'name': 'Estonia', 'capital': 'Tallinn'},
{'timezones': ['Africa/Cairo'], 'code': 'EG', 'continent': 'Africa', 'name': 'Egypt', 'capital': 'Cairo'},
{'timezones': ['Africa/Asmera'], 'code': 'ER', 'continent': 'Africa', 'name': 'Eritrea', 'capital': 'Asmara'},
{'timezones': ['Africa/Addis_Ababa'], 'code': 'ET', 'continent': 'Africa', 'name': 'Ethiopia', 'capital': 'Addis Ababa'},
{'timezones': ['Europe/Helsinki'], 'code': 'FI', 'continent': 'Europe', 'name': 'Finland', 'capital': 'Helsinki'},
{'timezones': ['Pacific/Fiji'], 'code': 'FJ', 'continent': 'Oceania', 'name': 'Fiji', 'capital': 'Suva'},
{'timezones': ['Europe/Paris'], 'code': 'FR', 'continent': 'Europe', 'name': 'France', 'capital': 'Paris'},
{'timezones': ['Africa/Libreville'], 'code': 'GA', 'continent': 'Africa', 'name': 'Gabon', 'capital': 'Libreville'},
{'timezones': ['Asia/Tbilisi'], 'code': 'GE', 'continent': 'Asia', 'name': 'Georgia', 'capital': 'Tbilisi'},
{'timezones': ['Africa/Accra'], 'code': 'GH', 'continent': 'Africa', 'name': 'Ghana', 'capital': 'Accra'},
{'timezones': ['Africa/Banjul'], 'code': 'GM', 'continent': 'Africa', 'name': 'The Gambia', 'capital': 'Banjul'},
{'timezones': ['Africa/Conakry'], 'code': 'GN', 'continent': 'Africa', 'name': 'Guinea', 'capital': 'Conakry'},
{'timezones': ['Europe/Athens'], 'code': 'GR', 'continent': 'Europe', 'name': 'Greece', 'capital': 'Athens'},
{'timezones': ['America/Guatemala'], 'code': 'GT', 'continent': 'North America', 'name': 'Guatemala', 'capital': 'Guatemala City'},
{'timezones': ['America/Guatemala'], 'code': 'GT', 'continent': 'North America', 'name': 'Haiti', 'capital': 'Port-au-Prince'},
{'timezones': ['Africa/Bissau'], 'code': 'GW', 'continent': 'Africa', 'name': 'Guinea-Bissau', 'capital': 'Bissau'},
{'timezones': ['America/Guyana'], 'code': 'GY', 'continent': 'South America', 'name': 'Guyana', 'capital': 'Georgetown'},
{'timezones': ['America/Tegucigalpa'], 'code': 'HN', 'continent': 'North America', 'name': 'Honduras', 'capital': 'Tegucigalpa'},
{'timezones': ['Europe/Budapest'], 'code': 'HU', 'continent': 'Europe', 'name': 'Hungary', 'capital': 'Budapest'},
{'timezones': ['Asia/Jakarta', 'Asia/Pontianak', 'Asia/Makassar', 'Asia/Jayapura'], 'code': 'ID', 'continent': 'Asia', 'name': 'Indonesia', 'capital': 'Jakarta'},
{'timezones': ['Europe/Dublin'], 'code': 'IE', 'continent': 'Europe', 'name': 'Republic of Ireland', 'capital': 'Dublin'},
{'timezones': ['Asia/Jerusalem'], 'code': 'IL', 'continent': 'Asia', 'name': 'Israel', 'capital': 'Jerusalem'},
{'timezones': ['Asia/Calcutta'], 'code': 'IN', 'continent': 'Asia', 'name': 'India', 'capital': 'New Delhi'},
{'timezones': ['Asia/Baghdad'], 'code': 'IQ', 'continent': 'Asia', 'name': 'Iraq', 'capital': 'Baghdad'},
{'timezones': ['Asia/Tehran'], 'code': 'IR', 'continent': 'Asia', 'name': 'Iran', 'capital': 'Tehran'},
{'timezones': ['Atlantic/Reykjavik'], 'code': 'IS', 'continent': 'Europe', 'name': 'Iceland', 'capital': 'Reykjav\xc3\xadk'},
{'timezones': ['Europe/Rome'], 'code': 'IT', 'continent': 'Europe', 'name': 'Italy', 'capital': 'Rome'},
{'timezones': ['America/Jamaica'], 'code': 'JM', 'continent': 'North America', 'name': 'Jamaica', 'capital': 'Kingston'},
{'timezones': ['Asia/Amman'], 'code': 'JO', 'continent': 'Asia', 'name': 'Jordan', 'capital': 'Amman'},
{'timezones': ['Asia/Tokyo'], 'code': 'JP', 'continent': 'Asia', 'name': 'Japan', 'capital': 'Tokyo'},
{'timezones': ['Africa/Nairobi'], 'code': 'KE', 'continent': 'Africa', 'name': 'Kenya', 'capital': 'Nairobi'},
{'timezones': ['Asia/Bishkek'], 'code': 'KG', 'continent': 'Asia', 'name': 'Kyrgyzstan', 'capital': 'Bishkek'},
{'timezones': ['Pacific/Tarawa', 'Pacific/Enderbury', 'Pacific/Kiritimati'], 'code': 'KI', 'continent': 'Oceania', 'name': 'Kiribati', 'capital': 'Tarawa'},
{'timezones': ['Asia/Pyongyang'], 'code': 'KP', 'continent': 'Asia', 'name': 'North Korea', 'capital': 'Pyongyang'},
{'timezones': ['Asia/Seoul'], 'code': 'KR', 'continent': 'Asia', 'name': 'South Korea', 'capital': 'Seoul'},
{'timezones': ['Asia/Kuwait'], 'code': 'KW', 'continent': 'Asia', 'name': 'Kuwait', 'capital': 'Kuwait City'},
{'timezones': ['Asia/Beirut'], 'code': 'LB', 'continent': 'Asia', 'name': 'Lebanon', 'capital': 'Beirut'},
{'timezones': ['Europe/Vaduz'], 'code': 'LI', 'continent': 'Europe', 'name': 'Liechtenstein', 'capital': 'Vaduz'},
{'timezones': ['Africa/Monrovia'], 'code': 'LR', 'continent': 'Africa', 'name': 'Liberia', 'capital': 'Monrovia'},
{'timezones': ['Africa/Maseru'], 'code': 'LS', 'continent': 'Africa', 'name': 'Lesotho', 'capital': 'Maseru'},
{'timezones': ['Europe/Vilnius'], 'code': 'LT', 'continent': 'Europe', 'name': 'Lithuania', 'capital': 'Vilnius'},
{'timezones': ['Europe/Luxembourg'], 'code': 'LU', 'continent': 'Europe', 'name': 'Luxembourg', 'capital': 'Luxembourg City'},
{'timezones': ['Europe/Riga'], 'code': 'LV', 'continent': 'Europe', 'name': 'Latvia', 'capital': 'Riga'},
{'timezones': ['Africa/Tripoli'], 'code': 'LY', 'continent': 'Africa', 'name': 'Libya', 'capital': 'Tripoli'},
{'timezones': ['Indian/Antananarivo'], 'code': 'MG', 'continent': 'Africa', 'name': 'Madagascar', 'capital': 'Antananarivo'},
{'timezones': ['Pacific/Majuro', 'Pacific/Kwajalein'], 'code': 'MH', 'continent': 'Oceania', 'name': 'Marshall Islands', 'capital': 'Majuro'},
{'timezones': ['Europe/Skopje'], 'code': 'MK', 'continent': 'Europe', 'name': 'Macedonia', 'capital': 'Skopje'},
{'timezones': ['Africa/Bamako'], 'code': 'ML', 'continent': 'Africa', 'name': 'Mali', 'capital': 'Bamako'},
{'timezones': ['Asia/Rangoon'], 'code': 'MM', 'continent': 'Asia', 'name': 'Myanmar', 'capital': 'Naypyidaw'},
{'timezones': ['Asia/Ulaanbaatar', 'Asia/Hovd', 'Asia/Choibalsan'], 'code': 'MN', 'continent': 'Asia', 'name': 'Mongolia', 'capital': 'Ulaanbaatar'},
{'timezones': ['Africa/Nouakchott'], 'code': 'MR', 'continent': 'Africa', 'name': 'Mauritania', 'capital': 'Nouakchott'},
{'timezones': ['Europe/Malta'], 'code': 'MT', 'continent': 'Europe', 'name': 'Malta', 'capital': 'Valletta'},
{'timezones': ['Indian/Mauritius'], 'code': 'MU', 'continent': 'Africa', 'name': 'Mauritius', 'capital': 'Port Louis'},
{'timezones': ['Indian/Maldives'], 'code': 'MV', 'continent': 'Asia', 'name': 'Maldives', 'capital': 'Mal\xc3\xa9'},
{'timezones': ['Africa/Blantyre'], 'code': 'MW', 'continent': 'Africa', 'name': 'Malawi', 'capital': 'Lilongwe'},
{'timezones': ['America/Mexico_City', 'America/Cancun', 'America/Merida', 'America/Monterrey', 'America/Mazatlan', 'America/Chihuahua', 'America/Hermosillo', 'America/Tijuana'], 'code': 'MX', 'continent': 'North America', 'name': 'Mexico', 'capital': 'Mexico City'},
{'timezones': ['Asia/Kuala_Lumpur', 'Asia/Kuching'], 'code': 'MY', 'continent': 'Asia', 'name': 'Malaysia', 'capital': 'Kuala Lumpur'},
{'timezones': ['Africa/Maputo'], 'code': 'MZ', 'continent': 'Africa', 'name': 'Mozambique', 'capital': 'Maputo'},
{'timezones': ['Africa/Windhoek'], 'code': 'NA', 'continent': 'Africa', 'name': 'Namibia', 'capital': 'Windhoek'},
{'timezones': ['Africa/Niamey'], 'code': 'NE', 'continent': 'Africa', 'name': 'Niger', 'capital': 'Niamey'},
{'timezones': ['Africa/Lagos'], 'code': 'NG', 'continent': 'Africa', 'name': 'Nigeria', 'capital': 'Abuja'},
{'timezones': ['America/Managua'], 'code': 'NI', 'continent': 'North America', 'name': 'Nicaragua', 'capital': 'Managua'},
{'timezones': ['Europe/Amsterdam'], 'code': 'NL', 'continent': 'Europe', 'name': 'Kingdom of the Netherlands', 'capital': 'Amsterdam'},
{'timezones': ['Europe/Oslo'], 'code': 'NO', 'continent': 'Europe', 'name': 'Norway', 'capital': 'Oslo'},
{'timezones': ['Asia/Katmandu'], 'code': 'NP', 'continent': 'Asia', 'name': 'Nepal', 'capital': 'Kathmandu'},
{'timezones': ['Pacific/Nauru'], 'code': 'NR', 'continent': 'Oceania', 'name': 'Nauru', 'capital': 'Yaren'},
{'timezones': ['Pacific/Auckland', 'Pacific/Chatham'], 'code': 'NZ', 'continent': 'Oceania', 'name': 'New Zealand', 'capital': 'Wellington'},
{'timezones': ['Asia/Muscat'], 'code': 'OM', 'continent': 'Asia', 'name': 'Oman', 'capital': 'Muscat'},
{'timezones': ['America/Panama'], 'code': 'PA', 'continent': 'North America', 'name': 'Panama', 'capital': 'Panama City'},
{'timezones': ['America/Lima'], 'code': 'PE', 'continent': 'South America', 'name': 'Peru', 'capital': 'Lima'},
{'timezones': ['Pacific/Port_Moresby'], 'code': 'PG', 'continent': 'Oceania', 'name': 'Papua New Guinea', 'capital': 'Port Moresby'},
{'timezones': ['Asia/Manila'], 'code': 'PH', 'continent': 'Asia', 'name': 'Philippines', 'capital': 'Manila'},
{'timezones': ['Asia/Karachi'], 'code': 'PK', 'continent': 'Asia', 'name': 'Pakistan', 'capital': 'Islamabad'},
{'timezones': ['Europe/Warsaw'], 'code': 'PL', 'continent': 'Europe', 'name': 'Poland', 'capital': 'Warsaw'},
{'timezones': ['Europe/Lisbon', 'Atlantic/Madeira', 'Atlantic/Azores'], 'code': 'PT', 'continent': 'Europe', 'name': 'Portugal', 'capital': 'Lisbon'},
{'timezones': ['Pacific/Palau'], 'code': 'PW', 'continent': 'Oceania', 'name': 'Palau', 'capital': 'Ngerulmud'},
{'timezones': ['America/Asuncion'], 'code': 'PY', 'continent': 'South America', 'name': 'Paraguay', 'capital': 'Asunci\xc3\xb3n'},
{'timezones': ['Asia/Qatar'], 'code': 'QA', 'continent': 'Asia', 'name': 'Qatar', 'capital': 'Doha'},
{'timezones': ['Europe/Bucharest'], 'code': 'RO', 'continent': 'Europe', 'name': 'Romania', 'capital': 'Bucharest'},
{'timezones': ['Europe/Kaliningrad', 'Europe/Moscow', 'Europe/Volgograd', 'Europe/Samara', 'Asia/Yekaterinburg', 'Asia/Omsk', 'Asia/Novosibirsk', 'Asia/Krasnoyarsk', 'Asia/Irkutsk', 'Asia/Yakutsk', 'Asia/Vladivostok', 'Asia/Sakhalin', 'Asia/Magadan', 'Asia/Kamchatka', 'Asia/Anadyr'], 'code': 'RU', 'continent': 'Europe', 'name': 'Russia', 'capital': 'Moscow'},
{'timezones': ['Africa/Kigali'], 'code': 'RW', 'continent': 'Africa', 'name': 'Rwanda', 'capital': 'Kigali'},
{'timezones': ['Asia/Riyadh'], 'code': 'SA', 'continent': 'Asia', 'name': 'Saudi Arabia', 'capital': 'Riyadh'},
{'timezones': ['Pacific/Guadalcanal'], 'code': 'SB', 'continent': 'Oceania', 'name': 'Solomon Islands', 'capital': 'Honiara'},
{'timezones': ['Indian/Mahe'], 'code': 'SC', 'continent': 'Africa', 'name': 'Seychelles', 'capital': 'Victoria'},
{'timezones': ['Africa/Khartoum'], 'code': 'SD', 'continent': 'Africa', 'name': 'Sudan', 'capital': 'Khartoum'},
{'timezones': ['Europe/Stockholm'], 'code': 'SE', 'continent': 'Europe', 'name': 'Sweden', 'capital': 'Stockholm'},
{'timezones': ['Asia/Singapore'], 'code': 'SG', 'continent': 'Asia', 'name': 'Singapore', 'capital': 'Singapore'},
{'timezones': ['Europe/Ljubljana'], 'code': 'SI', 'continent': 'Europe', 'name': 'Slovenia', 'capital': 'Ljubljana'},
{'timezones': ['Europe/Bratislava'], 'code': 'SK', 'continent': 'Europe', 'name': 'Slovakia', 'capital': 'Bratislava'},
{'timezones': ['Africa/Freetown'], 'code': 'SL', 'continent': 'Africa', 'name': 'Sierra Leone', 'capital': 'Freetown'},
{'timezones': ['Europe/San_Marino'], 'code': 'SM', 'continent': 'Europe', 'name': 'San Marino', 'capital': 'San Marino'},
{'timezones': ['Africa/Dakar'], 'code': 'SN', 'continent': 'Africa', 'name': 'Senegal', 'capital': 'Dakar'},
{'timezones': ['Africa/Mogadishu'], 'code': 'SO', 'continent': 'Africa', 'name': 'Somalia', 'capital': 'Mogadishu'},
{'timezones': ['America/Paramaribo'], 'code': 'SR', 'continent': 'South America', 'name': 'Suriname', 'capital': 'Paramaribo'},
{'timezones': ['Africa/Sao_Tome'], 'code': 'ST', 'continent': 'Africa', 'name': 'S\xc3\xa3o Tom\xc3\xa9 and Pr\xc3\xadncipe', 'capital': 'S\xc3\xa3o Tom\xc3\xa9'},
{'timezones': ['Asia/Damascus'], 'code': 'SY', 'continent': 'Asia', 'name': 'Syria', 'capital': 'Damascus'},
{'timezones': ['Africa/Lome'], 'code': 'TG', 'continent': 'Africa', 'name': 'Togo', 'capital': 'Lom\xc3\xa9'},
{'timezones': ['Asia/Bangkok'], 'code': 'TH', 'continent': 'Asia', 'name': 'Thailand', 'capital': 'Bangkok'},
{'timezones': ['Asia/Dushanbe'], 'code': 'TJ', 'continent': 'Asia', 'name': 'Tajikistan', 'capital': 'Dushanbe'},
{'timezones': ['Asia/Ashgabat'], 'code': 'TM', 'continent': 'Asia', 'name': 'Turkmenistan', 'capital': 'Ashgabat'},
{'timezones': ['Africa/Tunis'], 'code': 'TN', 'continent': 'Africa', 'name': 'Tunisia', 'capital': 'Tunis'},
{'timezones': ['Pacific/Tongatapu'], 'code': 'TO', 'continent': 'Oceania', 'name': 'Tonga', 'capital': 'Nuku\xca\xbbalofa'},
{'timezones': ['Europe/Istanbul'], 'code': 'TR', 'continent': 'Asia', 'name': 'Turkey', 'capital': 'Ankara'},
{'timezones': ['America/Port_of_Spain'], 'code': 'TT', 'continent': 'North America', 'name': 'Trinidad and Tobago', 'capital': 'Port of Spain'},
{'timezones': ['Pacific/Funafuti'], 'code': 'TV', 'continent': 'Oceania', 'name': 'Tuvalu', 'capital': 'Funafuti'},
{'timezones': ['Africa/Dar_es_Salaam'], 'code': 'TZ', 'continent': 'Africa', 'name': 'Tanzania', 'capital': 'Dodoma'},
{'timezones': ['Europe/Kiev', 'Europe/Uzhgorod', 'Europe/Zaporozhye', 'Europe/Simferopol'], 'code': 'UA', 'continent': 'Europe', 'name': 'Ukraine', 'capital': 'Kiev'},
{'timezones': ['Africa/Kampala'], 'code': 'UG', 'continent': 'Africa', 'name': 'Uganda', 'capital': 'Kampala'},
{'timezones': ['America/New_York', 'America/Detroit', 'America/Kentucky/Louisville', 'America/Kentucky/Monticello', 'America/Indiana/Indianapolis', 'America/Indiana/Marengo', 'America/Indiana/Knox', 'America/Indiana/Vevay', 'America/Chicago', 'America/Indiana/Vincennes', 'America/Indiana/Petersburg', 'America/Menominee', 'America/North_Dakota/Center', 'America/North_Dakota/New_Salem', 'America/Denver', 'America/Boise', 'America/Shiprock', 'America/Phoenix', 'America/Los_Angeles', 'America/Anchorage', 'America/Juneau', 'America/Yakutat', 'America/Nome', 'America/Adak', 'Pacific/Honolulu'], 'code': 'US', 'continent': 'North America', 'name': 'United States', 'capital': 'Washington, D.C.'},
{'timezones': ['America/Montevideo'], 'code': 'UY', 'continent': 'South America', 'name': 'Uruguay', 'capital': 'Montevideo'},
{'timezones': ['Asia/Samarkand', 'Asia/Tashkent'], 'code': 'UZ', 'continent': 'Asia', 'name': 'Uzbekistan', 'capital': 'Tashkent'},
{'timezones': ['Europe/Vatican'], 'code': 'VA', 'continent': 'Europe', 'name': 'Vatican City', 'capital': 'Vatican City'},
{'timezones': ['America/Caracas'], 'code': 'VE', 'continent': 'South America', 'name': 'Venezuela', 'capital': 'Caracas'},
{'timezones': ['Asia/Saigon'], 'code': 'VN', 'continent': 'Asia', 'name': 'Vietnam', 'capital': 'Hanoi'},
{'timezones': ['Pacific/Efate'], 'code': 'VU', 'continent': 'Oceania', 'name': 'Vanuatu', 'capital': 'Port Vila'},
{'timezones': ['Asia/Aden'], 'code': 'YE', 'continent': 'Asia', 'name': 'Yemen', 'capital': "Sana'a"},
{'timezones': ['Africa/Lusaka'], 'code': 'ZM', 'continent': 'Africa', 'name': 'Zambia', 'capital': 'Lusaka'},
{'timezones': ['Africa/Harare'], 'code': 'ZW', 'continent': 'Africa', 'name': 'Zimbabwe', 'capital': 'Harare'},
{'timezones': ['Africa/Algiers'], 'code': 'DZ', 'continent': 'Africa', 'name': 'Algeria', 'capital': 'Algiers'},
{'timezones': ['Europe/Sarajevo'], 'code': 'BA', 'continent': 'Europe', 'name': 'Bosnia and Herzegovina', 'capital': 'Sarajevo'},
{'timezones': ['Asia/Phnom_Penh'], 'code': 'KH', 'continent': 'Asia', 'name': 'Cambodia', 'capital': 'Phnom Penh'},
{'timezones': ['Africa/Bangui'], 'code': 'CF', 'continent': 'Africa', 'name': 'Central African Republic', 'capital': 'Bangui'},
{'timezones': ['Africa/Ndjamena'], 'code': 'TD', 'continent': 'Africa', 'name': 'Chad', 'capital': "N'Djamena"},
{'timezones': ['Indian/Comoro'], 'code': 'KM', 'continent': 'Africa', 'name': 'Comoros', 'capital': 'Moroni'},
{'timezones': ['Europe/Zagreb'], 'code': 'HR', 'continent': 'Europe', 'name': 'Croatia', 'capital': 'Zagreb'},
{'timezones': ['Asia/Dili'], 'code': 'TL', 'continent': 'Asia', 'name': 'East Timor', 'capital': 'Dili'},
{'timezones': ['America/El_Salvador'], 'code': 'SV', 'continent': 'North America', 'name': 'El Salvador', 'capital': 'San Salvador'},
{'timezones': ['Africa/Malabo'], 'code': 'GQ', 'continent': 'Africa', 'name': 'Equatorial Guinea', 'capital': 'Malabo'},
{'timezones': ['America/Grenada'], 'code': 'GD', 'continent': 'North America', 'name': 'Grenada', 'capital': "St. George's"},
{'timezones': ['Asia/Almaty', 'Asia/Qyzylorda', 'Asia/Aqtobe', 'Asia/Aqtau', 'Asia/Oral'], 'code': 'KZ', 'continent': 'Asia', 'name': 'Kazakhstan', 'capital': 'Astana'},
{'timezones': ['Asia/Vientiane'], 'code': 'LA', 'continent': 'Asia', 'name': 'Laos', 'capital': 'Vientiane'},
{'timezones': ['Pacific/Truk', 'Pacific/Ponape', 'Pacific/Kosrae'], 'code': 'FM', 'continent': 'Oceania', 'name': 'Federated States of Micronesia', 'capital': 'Palikir'},
{'timezones': ['Europe/Chisinau'], 'code': 'MD', 'continent': 'Europe', 'name': 'Moldova', 'capital': 'Chi\xc5\x9fin\xc4\x83u'},
{'timezones': ['Europe/Monaco'], 'code': 'MC', 'continent': 'Europe', 'name': 'Monaco', 'capital': 'Monaco'},
{'timezones': ['Europe/Podgorica'], 'code': 'ME', 'continent': 'Europe', 'name': 'Montenegro', 'capital': 'Podgorica'},
{'timezones': ['Africa/Casablanca'], 'code': 'MA', 'continent': 'Africa', 'name': 'Morocco', 'capital': 'Rabat'},
{'timezones': ['America/St_Kitts'], 'code': 'KN', 'continent': 'North America', 'name': 'Saint Kitts and Nevis', 'capital': 'Basseterre'},
{'timezones': ['America/St_Lucia'], 'code': 'LC', 'continent': 'North America', 'name': 'Saint Lucia', 'capital': 'Castries'},
{'timezones': ['America/St_Vincent'], 'code': 'VC', 'continent': 'North America', 'name': 'Saint Vincent and the Grenadines', 'capital': 'Kingstown'},
{'timezones': ['Pacific/Apia'], 'code': 'WS', 'continent': 'Oceania', 'name': 'Samoa', 'capital': 'Apia'},
{'timezones': ['Europe/Belgrade'], 'code': 'RS', 'continent': 'Europe', 'name': 'Serbia', 'capital': 'Belgrade'},
{'timezones': ['Africa/Johannesburg'], 'code': 'ZA', 'continent': 'Africa', 'name': 'South Africa', 'capital': 'Pretoria'},
{'timezones': ['Europe/Madrid', 'Africa/Ceuta', 'Atlantic/Canary'], 'code': 'ES', 'continent': 'Europe', 'name': 'Spain', 'capital': 'Madrid'},
{'timezones': ['Asia/Colombo'], 'code': 'LK', 'continent': 'Asia', 'name': 'Sri Lanka', 'capital': 'Sri Jayewardenepura Kotte'},
{'timezones': ['Africa/Mbabane'], 'code': 'SZ', 'continent': 'Africa', 'name': 'Swaziland', 'capital': 'Mbabane'},
{'timezones': ['Europe/Zurich'], 'code': 'CH', 'continent': 'Europe', 'name': 'Switzerland', 'capital': 'Bern'},
{'timezones': ['Asia/Dubai'], 'code': 'AE', 'continent': 'Asia', 'name': 'United Arab Emirates', 'capital': 'Abu Dhabi'},
{'timezones': ['Europe/London'], 'code': 'GB', 'continent': 'Europe', 'name': 'United Kingdom', 'capital': 'London'},
]

In [489]:
countries[101:150]

[{'capital': 'Kuala Lumpur',
  'code': 'MY',
  'continent': 'Asia',
  'name': 'Malaysia',
  'timezones': ['Asia/Kuala_Lumpur', 'Asia/Kuching']},
 {'capital': 'Maputo',
  'code': 'MZ',
  'continent': 'Africa',
  'name': 'Mozambique',
  'timezones': ['Africa/Maputo']},
 {'capital': 'Windhoek',
  'code': 'NA',
  'continent': 'Africa',
  'name': 'Namibia',
  'timezones': ['Africa/Windhoek']},
 {'capital': 'Niamey',
  'code': 'NE',
  'continent': 'Africa',
  'name': 'Niger',
  'timezones': ['Africa/Niamey']},
 {'capital': 'Abuja',
  'code': 'NG',
  'continent': 'Africa',
  'name': 'Nigeria',
  'timezones': ['Africa/Lagos']},
 {'capital': 'Managua',
  'code': 'NI',
  'continent': 'North America',
  'name': 'Nicaragua',
  'timezones': ['America/Managua']},
 {'capital': 'Amsterdam',
  'code': 'NL',
  'continent': 'Europe',
  'name': 'Kingdom of the Netherlands',
  'timezones': ['Europe/Amsterdam']},
 {'capital': 'Oslo',
  'code': 'NO',
  'continent': 'Europe',
  'name': 'Norway',
  'timezones'

In [490]:
country_capital_city_tz = dict()
[country_capital_city_tz.update({name['name']:name['continent'].split()[-1]+"/"+"_".join(name['capital'].split(" ")).replace(",","")}) for name in countries]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [491]:
country_capital_city_tz['France']

'Europe/Paris'

In [492]:
country_capital_city_tz['United States']

'America/Washington_D.C.'

In [493]:
import json
with open('country_capital_city_tz.json', 'w') as file:
     file.write(json.dumps(country_capital_city_tz))

In [494]:
!head country_capital_city_tz.json


{"Canada": "America/Ottawa", "East Timor": "Asia/Dili", "Turkmenistan": "Asia/Ashgabat", "Lithuania": "Europe/Vilnius", "Cambodia": "Asia/Phnom_Penh", "Saint Kitts and Nevis": "America/Basseterre", "Ethiopia": "Africa/Addis_Ababa", "The Gambia": "Africa/Banjul", "Sri Lanka": "Asia/Sri_Jayewardenepura_Kotte", "Swaziland": "Africa/Mbabane", "Argentina": "America/Buenos_Aires", "Bolivia": "America/Sucre", "Cameroon": "Africa/Yaound\u00e9", "Burkina Faso": "Africa/Ouagadougou", "Ghana": "Africa/Accra", "Saudi Arabia": "Asia/Riyadh", "Laos": "Asia/Vientiane", "Japan": "Asia/Tokyo", "Republic of Ireland": "Europe/Dublin", "Slovenia": "Europe/Ljubljana", "Guatemala": "America/Guatemala_City", "Bosnia and Herzegovina": "Europe/Sarajevo", "Kuwait": "Asia/Kuwait_City", "Germany": "Europe/Berlin", "Dominica": "America/Roseau", "Liberia": "Africa/Monrovia", "Maldives": "Asia/Mal\u00e9", "Pakistan": "Asia/Islamabad", "Oman": "Asia/Muscat", "Tanzania": "Africa/Dodoma", "Seychelles": "Africa/Victoria

### Add TimeZone information for each session )

### SWITCHING TO Map Reduce

In [495]:
!pwd

/data_data/session_length


In [507]:
%%writefile MRtranslate.py

import mrjob
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob import step
from mrjob.protocol import RawProtocol

from collections import defaultdict, Counter

import re
import sys
import json

import datetime
from datetime import datetime
from dateutil import parser
from dateutil import tz
import pytz

import ast

import goslate 
import translate
from googletrans import Translator

import requests


class MRtranslate(MRJob):

    def __init__(self, *args, **kwargs):
        super(MRtranslate, self).__init__(*args, **kwargs)
        self.min_line_length = 12
        self.translator   = Translator()
        self.day_of_week  = {0:"Monday",1:"Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"}
        self.time_of_day  = {"Night":[0,1,2,3,4], "Dawn":[5,6],"Morning":[7,8,9],"LateMorning":[10,11], "Lunch":[12,13],"Afternoon":[14,15,16],"LateAfternoon":[17,18],"Evening":[19,20,21],"LateEvening":[22,23]}

    def configure_options(self):
        self.add_file_option('--tzFile', default=None, dest='tzFile')
        
    def translate(self, text):
        return self.translator.translate(text)
    
    def utc_to_local(self, utc_timestamp, local_tz):
        try:
            local_dt = utc_timestamp.replace(tzinfo=pytz.utc).astimezone(local_tz)
            #sys.stderr.write("LOCAL_DT:"+repr(local_dt)+"\n")
            result = local_tz.normalize(local_dt)
            #sys.stderr.write("RESULT ====> "+repr(result)+"\n")
        except:
            result = None
        return result
    
    def get_time_of_day(self, hour):
        for key in self.time_of_day.keys():
            if hour in self.time_of_day[key]:
                return key

    def mapper_translate(self, _, line_):
    
        if len(line_) > self.min_line_length:
            
            line   = line_.split("\t")
            user_id    = list(line)[0]
            values = line[1:]

            yield user_id, values
        
        
    def reducer_timezone(self, user_id, values):
        
        tzFile = str(self.options.tzFile) 
        
        tz_dict = dict()
        
        with open(tzFile,'r') as f:
            
            lines = f.readlines()
            
            tz_dict = ast.literal_eval(lines[0])
        
        for value in values:

                utc_time  = value[0][:-1]
                traid     = value[1]
                artname   = self.translator.translate(value[2]).text
                something = value[3]
                tracname  = self.translator.translate(value[4]).text
                session_window = value[5]
                gender    = value[6]
                age       = value[7]
                country   = value[8]
                registered= value[9] 
                reg_date  = value[10]
                
                try:
                    local_tz  = pytz.timezone(tz_dict[country])
                except:
                    local_tz  = None
                try:
                    utc    = datetime.strptime(utc_time, '%Y-%m-%dT%H:%M:%S')
                except: 
                    utc    = None
                
                try:
                    event_time = self.utc_to_local(utc, local_tz)
                except: 
                    event_time = None
                    
                try:
                    time_diff  = event_time.utcoffset()
                    local_time = event_time + time_diff
                except:
                    local_time = "Null"
                
                try:
                    day_of_week = self.day_of_week[local_time.weekday()]
                except:
                    day_of_week = "Null"
                    
                try:
                    time_of_day = self.get_time_of_day(local_time.hour)
                except:
                    time_of_day = "Null"
                    
                value_ = "\t".join([str(x) for x in (utc_time, traid, artname, something, tracname, session_window, gender, age, country, registered, reg_date, str(local_tz), str(local_time), str(time_of_day), str(day_of_week))])

                yield user_id, value_
 




    def steps(self):
        
        JOBCONF = {
            'mapred.map.tasks': 10,
            'mapred.reduce.tasks': 10,
            'mapreduce.job.local.dir':'/data_data/session_length/hadoop',
            'mapred.task.tmp.dir':'/data_data/session_length/hadoop',
            'mapreduce.task.tmp.dir':'/data_data/session_length/hadoop',
            'mapreduce.output.fileoutputformat.outputdir': '/data_data/session_length/hadoop',
            'mapreduce.cluster.local.dir':'/data_data/session_length/hadoop',
            'mapred.partition.keypartitioner.options': '-k1',
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k1',
            'mapreduce.map.memory.mb':40960,
            'mapreduce.reduce.memory.mb':40960,
            'mapreduce.map.java.opts':'-Xmx30720m',
            'mapreduce.reduce.java.opts':'-Xmx61440m'
        }
        JOBCONF1 = {
            'mapred.map.tasks': 10,
            'mapred.reduce.tasks': 10,
            'mapreduce.job.local.dir':'/data_data/session_length/hadoop',
            'mapred.task.tmp.dir':'/data_data/session_length/hadoop',
            'mapreduce.task.tmp.dir':'/data_data/session_length/hadoop',
            'mapreduce.output.fileoutputformat.outputdir': '/data_data/session_length/hadoop',
            'mapreduce.cluster.local.dir':'/data_data/session_length/hadoop',
            'mapred.partition.keypartitioner.options': '-k1',
            'mapred.output.key.comparator.class': 'org.apache.hadoop.mapred.lib.KeyFieldBasedComparator',
            'mapred.text.key.comparator.options': '-k1',
            'mapreduce.map.memory.mb':40960,
            'mapreduce.reduce.memory.mb':40960,
            'mapreduce.map.java.opts':'-Xmx30720m',
            'mapreduce.reduce.java.opts':'-Xmx61440m'
        }
        
        return [MRStep(mapper=self.mapper_translate,
                      reducer=self.reducer_timezone)]
       

if __name__ == '__main__':
    MRtranslate.run()


Overwriting MRtranslate.py


In [508]:
!ls -alrth  data/Session_data_*

-rw-r--r-- 1 hduser hadoop 153K Sep 16 11:15 data/Session_data_1k.csv
-rw-r--r-- 1 hduser hadoop 1.6M Sep 16 11:15 data/Session_data_10k.csv
-rw-r--r-- 1 hduser hadoop  17M Sep 16 11:15 data/Session_data_100k.csv
-rw-r--r-- 1 hduser hadoop 1.4K Sep 16 12:45 data/Session_data_10.csv
-rw-r--r-- 1 hduser hadoop 164M Sep 16 20:17 data/Session_data_1M.csv


In [511]:
!rm /data_data/session_length/session_data_10.tsv
#!rm -R /tmp/MRtranslate*
!python MRtranslate.py ./data/Session_data_10.csv --tzFile=/data_data/session_length/country_capital_city_tz.json  > Session_data_10_translated.tsv

rm: cannot remove '/data_data/session_length/session_data_10.tsv': No such file or directory
/usr/local/lib/python2.7/dist-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)
No handlers could be found for logger "mrjob.launch"
No configs found; falling back on auto-configuration
No configs specified for inline runner
configure_options() is deprecated and will be removed in v0.7.0; please use configure_args() instead.
add_file_option() is deprecated and will be removed in v0.7.0. Use add_file_arg() instead.
Running step 1 of 1...
Creating temp directory /tmp/MRtranslate.hduser.20180916.202135.104208
configure_options() is deprecated and will be removed in v0.7.0; please use configure_args() instead.
add_file_option() is deprecated and will be removed in v0.7.0. Use add_file_arg() instead.
configure_options() is deprecated and will be removed in v0.7.0; please use con

#### Using HOLIDAY API to determine whether day is holiday in each country

https://holidayapi.com/

In [514]:
holidays_dict = {
"	Angola	":"	AO	",
"	Austria	":"	AT	",
"	Australia	":"	AU	",
"	Aruba	":"	AW	",
"	Falkland Islands	":"	AX	",
"	Bosnia and Herzegovina	":"	BA	",
"	Belgium	":"	BE	",
"	Bulgaria	":"	BG	",
"	Bolivia	":"	BO	",
"	Brazil	":"	BR	",
"	The Bahamas	":"	BS	",
"	Canada	":"	CA	",
"	Switzerland	":"	CH	",
"	China	":"	CN	",
"	Colombia	":"	CO	",
"	Costa Rica	":"	CR	",
"	Cuba	":"	CU	",
"	Czech Republic	":"	CZ	",
"	Germany	":"	DE	",
"	Denmark	":"	DK	",
"	Dominican Republic	":"	DO	",
"	Ecuador	":"	EC	",
"	Spain	":"	ES	",
"	Finland	":"	FI	",
"	France	":"	FR	",
"	Alsace	":"	FR	",
"	United Kingdom	":"	GB	",
"	England	":"	GB	",
"	Northern Ireland	":"	GB	",
"	Scotland	":"	GB	",
"	Wales	":"	GB	",
"	Greece	":"	GR	",
"	Guatemala	":"	GT	",
"	Hong Kong	":"	HK	",
"	Honduras	":"	HN	",
"	Croatia	":"	HR	",
"	Hungary	":"	HU	",
"	Indonesia	":"	ID	",
"	Ireland	":"	IE	",
"	India	":"	IN	",
"	Israel	":"	IL	",
"	Iceland	":"	IS	",
"	Italy	":"	IT	",
"	Japan	":"	JP	",
"	Kazakhstan	":"	KZ	",
"	Lesotho	":"	LS	",
"	Luxembourg	":"	LU	",
"	Madagascar	":"	MG	",
"	Martinique	":"	MQ	",
"	Malta	":"	MT	",
"	Mauritius	":"	MU	",
"	Mexico	":"	MX	",
"	Mozambique	":"	MZ	",
"	Nigeria	":"	NG	",
"	Netherlands	":"	NL	",
"	Norway	":"	NO	",
"	Peru	":"	PE	",
"	Pakistan	":"	PK	",
"	Philippines	":"	PH	",
"	Poland	":"	PL	",
"	Puerto Rico	":"	PR	",
"	Portugal	":"	PT	",
"	Paraguay	":"	PY	",
"	Reunion	":"	RE	",
"	Romania	":"	RO	",
"	Russia	":"	RU	",
"	Seychelles	":"	SC	",
"	Sweden	":"	SE	",
"	Singapore	":"	SG	",
"	Slovenia	":"	SI	",
"	Sao Tome and Principe	":"	ST	",
"	Slovakia	":"	SK	",
"	Tunisia	":"	TN	",
"	Turkey	":"	TR	",
"	Ukraine	":"	UA	",
"	United States	":"	US	",
"	Uruguay	":"	UY	",
"	Venezuela	":"	VE	",
"	South Africa	":"	ZA	"}

In [515]:
h_dict = dict()
holiday_dict = [h_dict.update({value.strip():key.strip()}) for value, key in holidays_dict.items()]
h_dict

{'Alsace': 'FR',
 'Angola': 'AO',
 'Aruba': 'AW',
 'Australia': 'AU',
 'Austria': 'AT',
 'Belgium': 'BE',
 'Bolivia': 'BO',
 'Bosnia and Herzegovina': 'BA',
 'Brazil': 'BR',
 'Bulgaria': 'BG',
 'Canada': 'CA',
 'China': 'CN',
 'Colombia': 'CO',
 'Costa Rica': 'CR',
 'Croatia': 'HR',
 'Cuba': 'CU',
 'Czech Republic': 'CZ',
 'Denmark': 'DK',
 'Dominican Republic': 'DO',
 'Ecuador': 'EC',
 'England': 'GB',
 'Falkland Islands': 'AX',
 'Finland': 'FI',
 'France': 'FR',
 'Germany': 'DE',
 'Greece': 'GR',
 'Guatemala': 'GT',
 'Honduras': 'HN',
 'Hong Kong': 'HK',
 'Hungary': 'HU',
 'Iceland': 'IS',
 'India': 'IN',
 'Indonesia': 'ID',
 'Ireland': 'IE',
 'Israel': 'IL',
 'Italy': 'IT',
 'Japan': 'JP',
 'Kazakhstan': 'KZ',
 'Lesotho': 'LS',
 'Luxembourg': 'LU',
 'Madagascar': 'MG',
 'Malta': 'MT',
 'Martinique': 'MQ',
 'Mauritius': 'MU',
 'Mexico': 'MX',
 'Mozambique': 'MZ',
 'Netherlands': 'NL',
 'Nigeria': 'NG',
 'Northern Ireland': 'GB',
 'Norway': 'NO',
 'Pakistan': 'PK',
 'Paraguay': 'PY',


#### API Usage

In [516]:
import holidayapi

In [517]:
holiday_api_key = 'c37751f8-3eea-4a41-b839-d6e49ae3c131'

In [518]:
hapi = holidayapi.v1(holiday_api_key)

parameters = {
    'country': 'US',
    'year':    2016,
    'month':    7,
    'day':      4
    }

holidays = hapi.holidays(parameters)

In [519]:
holidays

{u'holidays': [{u'date': u'2016-07-04',
   u'name': u'Independence Day',
   u'observed': u'2016-07-04',
   u'public': True}],
 u'status': 200}

Py4JJavaError: An error occurred while calling o115.showString.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange hashpartitioning(country#3, 200)
+- *(1) HashAggregate(keys=[country#3], functions=[], output=[country#3])
   +- *(1) FileScan csv [country#3] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/data_data/session_length/data/lastFM/lastfm-dataset-1K/userid-profile.tsv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<country:string>

	at org.apache.spark.sql.catalyst.errors.package$.attachTree(package.scala:56)
	at org.apache.spark.sql.execution.exchange.ShuffleExchangeExec.doExecute(ShuffleExchangeExec.scala:119)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:131)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$execute$1.apply(SparkPlan.scala:127)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.

TypeError: Column is not iterable

datetime.datetime(2009, 5, 5, 17, 8, 57, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>)

In [400]:
local_time

datetime.datetime(2009, 5, 5, 8, 8, 57, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>)

In [344]:
!tail session_data_10.tsv 

"user_000001"	"2009-05-04T23:08:57\tf1b1cf71-bd35-4e99-8624-24a6e15f133a\tDeep Dish\t\tFuck Me Im Famous (Pacha Ibiza)-09-28-2007\t23\tm\t\tJapan\tAug 13, 2006\t2006-08-13\tAsia/Tokyo\t2009-05-05 08:08:57+09:00"
"user_000001"	"2009-05-04T13:54:10\ta7f7df4a-77d8-4f12-8acd-5c60c93f4de8\tRyuichi Sakamoto\t\tComposition 0919 (Live_2009_4_15)\t13\tm\t\tJapan\tAug 13, 2006\t2006-08-13\tAsia/Tokyo\t2009-05-04 22:54:10+09:00"
"user_000001"	"2009-05-04T13:52:04\ta7f7df4a-77d8-4f12-8acd-5c60c93f4de8\tRyuichi Sakamoto\t\tMc2 (Live_2009_4_15)\t13\tm\t\tJapan\tAug 13, 2006\t2006-08-13\tAsia/Tokyo\t2009-05-04 22:52:04+09:00"
"user_000001"	"2009-05-04T13:42:52\ta7f7df4a-77d8-4f12-8acd-5c60c93f4de8\tRyuichi Sakamoto\t\tHibari (Live_2009_4_15)\t13\tm\t\tJapan\tAug 13, 2006\t2006-08-13\tAsia/Tokyo\t2009-05-04 22:42:52+09:00"
"user_000001"	"2009-05-04T13:42:11\ta7f7df4a-77d8-4f12-8acd-5c60c93f4de8\tRyuichi Sakamoto\t\tMc1 (Live_2009_4_15)\t13\tm\t\tJapan\tAug 13, 2006\t2006-08-13\tAsia/Tokyo\t2009-05-04 

In [214]:
!head session_data_10.csv

"user_000001"	"2009-05-04T23:08:57Z\tf1b1cf71-bd35-4e99-8624-24a6e15f133a\tDeep Dish\t\tFuck Me Im Famous (Pacha Ibiza)-09-28-2007\t23\tm\t\tJapan\tAug 13, 2006\t2006-08-13"
"user_000001"	"2009-05-04T13:54:10Z\ta7f7df4a-77d8-4f12-8acd-5c60c93f4de8\tRyuichi Sakamoto\t\tComposition 0919 (Live_2009_4_15)\t13\tm\t\tJapan\tAug 13, 2006\t2006-08-13"
"user_000001"	"2009-05-04T13:52:04Z\ta7f7df4a-77d8-4f12-8acd-5c60c93f4de8\tRyuichi Sakamoto\t\tMc2 (Live_2009_4_15)\t13\tm\t\tJapan\tAug 13, 2006\t2006-08-13"
"user_000001"	"2009-05-04T13:42:52Z\ta7f7df4a-77d8-4f12-8acd-5c60c93f4de8\tRyuichi Sakamoto\t\tHibari (Live_2009_4_15)\t13\tm\t\tJapan\tAug 13, 2006\t2006-08-13"
"user_000001"	"2009-05-04T13:42:11Z\ta7f7df4a-77d8-4f12-8acd-5c60c93f4de8\tRyuichi Sakamoto\t\tMc1 (Live_2009_4_15)\t13\tm\t\tJapan\tAug 13, 2006\t2006-08-13"
"user_000001"	"2009-05-04T13:38:31Z\ta7f7df4a-77d8-4f12-8acd-5c60c93f4de8\tRyuichi Sakamoto\t\tTo Stanford (Live_2009_4_15)\t13\tm\t\tJapan\tAug 13, 2006\t2006-08-13"
"user_0